In [1]:
from brew_features import *
bf = brewFeatures()

/home/ymiyamoto5/notebooks/brew_features.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/home/ymiyamoto5/notebooks/brew_features.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Loading BokehJS ...

In [4]:
app = bf.make_app()
app.run_server(mode='external', host='0.0.0.0', port=8050,debug=True)

Dash app running on http://0.0.0.0:8050/


In [3]:
disp_col = {}
disp_col['プレス荷重shift'] = [1,1]      # 項目を表示したい場所を[row,col]で指定
disp_col['右垂直'] = [1,1]
disp_col['スライド変位右'] = [2,1]
disp_col['スライド変位左'] = [2,1]
disp_col['M30ボルスタ右奥'] = [3,1]
disp_col['加速度左右_X_左+500G'] = [1,2]
bf.set_dispcol(disp_col)

In [13]:
disp_col = {}
disp_col['プレス荷重shift'] = [1,1,None]      # 項目を表示したい場所を[row,col]で指定
disp_col['右垂直'] = [1,1,None]
disp_col['スライド変位右'] = [2,1,None]
disp_col['スライド変位左'] = [2,1,None]
disp_col['変位左右差'] = [3,1,'df["スライド変位右"]-df["スライド変位左"]']
disp_col['加速度左右_X_左+500G'] = [1,2,None]
bf.set_dispcol(disp_col)

In [11]:
disp_col = {}
disp_col['プレス荷重shift'] = [1,1,None]
disp_col['dFdt'] = [2,1,'(df["プレス荷重shift"].diff() / df["time"].diff())']
disp_col['F*dFdt'] = [3,1,'df["プレス荷重shift"]*(df["プレス荷重shift"].diff() / df["time"].diff())']
disp_col['スライド変位右'] = [5,1,None]
disp_col['加速度左右_X_左+500G'] = [4,1,None]
bf.set_dispcol(disp_col)

In [2]:
# %reload_ext autoreload
# %autoreload 2
# from file_utils import *       # 独自関数群
# from misc_utils import *    # 独自関数群
from plotly_utils import *    # 独自関数群
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'  # レポート作成用
import os
import re
#sns.set(font=['osaka_unicode'])
np.set_printoptions(precision=20, suppress=True)
import matplotlib as mpl
mpl.rc('figure', facecolor='#eeeeee',figsize=(6.4,4.8))  # そのままpptに貼れるくらいのサイズ
mpl.rc('axes', facecolor='#ffffff',edgecolor="#dddddd")  # 背景白,ラベル部gray
mpl.rc('grid',color='#dddddd',linewidth=1)               # ラベル部と同色のグリッド
mpl.rc('axes', titlesize=18,labelsize=16,grid=True)
mpl.rc('xtick',labelsize=14)
mpl.rc('ytick',labelsize=14)
mpl.rc('legend',fontsize=14)
plt.rcParams['axes.axisbelow'] = True
cmap = plt.get_cmap("tab10")       # tableau colormap
import warnings
#warnings.filterwarnings('ignore')  # 最初から無視するのは良くないのでコメントアウトしておく
DATA_ROOT='/Users/hao/data/ADDQ/'

In [3]:
from pathlib import Path
p = Path(DATA_ROOT+'/20211004ブレークスルー/')
flist = list(sorted(p.glob('*.CSV')))

In [4]:
!ls /Users/hao/data/ADDQ/

20211004ブレークスルー	DATA


In [5]:
flist

[PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040002_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040003_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040004_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040005_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040007_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040009_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040010_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040011_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040012_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040013_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040014_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040016_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/2110040017_A0000.CSV'),
 PosixPath('/Users/hao/data/ADDQ/20211004ブレークスルー/21

In [6]:
def read_logger(f):
    # 9行目以降のメタ情報を読み込むために一旦8行目以降を読み込み
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7] )
    unit = df.iloc[4]
    offset = df.iloc[3]
    calibration = df.iloc[2]
    v_range = df.iloc[1]
    channel = df.iloc[0]
    # 改めて8行目をヘッダにして読み込む
    # df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1).set_index('time')
    # floatのindex作っちゃダメ!!!
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1)
    try:
        df['プレス荷重shift'] = df['プレス荷重'].shift(-640)
        df['dF/dt'] = df['プレス荷重shift'].diff()
        df['F*dF/dt'] = df['プレス荷重shift'] * df['dF/dt']
        if '加速度左右_X_左+' in df.columns:
            df = df.rename({'加速度左右_X_左+':'加速度左右_X_左+500G'},axis=1)
        if '加速度上下_Y_下+' in df.columns:
            df = df.rename({'加速度上下_Y_下+':'加速度上下_Y_下+500G'},axis=1)        
        if '加速度前後_Z_前+' in df.columns:
            df = df.rename({'加速度前後_Z_前+':'加速度前後_Z_前+500G'},axis=1)        
    except KeyError:
        print('プレス荷重　無し')

    return df,unit,offset,calibration,v_range,channel

In [7]:
f = flist[10]
df = read_logger(f)[0][:3000]
df

,time,右垂直,左垂直,加速度左右_X_左+500G,加速度上下_Y_下+500G,加速度前後_Z_前+500G,M30ボルスタ右奥,M20前左,M20前右,M20後左,M20後右,スライド＿金型隙間,プレス荷重,スライド変位左,スライド変位右,プレス荷重shift,dF/dt,F*dF/dt
0,0.00000,-0.812500,-2.640625,-0.108025,-0.121200,-0.079931,-5.896688,-26.208190,-0.633475,9.606306,-13.312160,0.000312,-0.171875,10.061410,10.057970,-0.343750,NaN,NaN
1,0.00005,-0.718750,-2.593750,-0.085675,-0.068175,-0.133219,-57.394430,-3.812100,2.375531,19.701070,-2.726587,0.001562,0.000000,10.061410,10.057970,-0.275000,0.068750,-0.018906
2,0.00010,-0.578125,-2.437500,-0.141550,-0.185587,-0.072319,-34.200790,-1.111863,0.633475,9.606306,5.613562,0.001249,-0.068750,10.061410,10.057970,-0.412500,-0.137500,0.056719
3,0.00015,-1.250000,-2.953125,-0.074500,-0.246188,-0.019031,27.910990,-15.407240,-5.067800,-8.466575,-5.613562,0.000937,-0.034375,10.061410,10.057970,-0.446875,-0.034375,0.015361
4,0.00020,-1.875000,-3.671875,-0.026075,-0.321937,-0.034256,43.242380,-14.454210,-6.651487,1.465369,-10.104410,0.000624,-0.034375,10.061410,10.057970,-0.275000,0.171875,-0.047266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.14975,-0.671875,-5.593750,0.044700,-0.677963,-0.121800,73.905150,-0.476513,-10.927440,-3.419194,3.207750,0.000937,5.431250,0.457656,0.455782,-0.343750,-0.034375,0.011816
2996,0.14980,-0.359375,-5.562500,-0.003725,-0.564338,-0.110381,29.483440,-0.476513,-12.827870,6.187112,3.207750,0.001249,5.018750,0.458750,0.457344,-0.240625,0.103125,-0.024814
2997,0.14985,-0.765625,-5.718750,0.070775,-0.552975,-0.057094,11.007150,6.194662,-13.302970,7.001206,1.443487,0.001562,4.846875,0.460312,0.458438,-0.103125,0.137500,-0.014180
2998,0.14990,-1.171875,-6.359375,0.081950,-0.685538,0.003806,40.883700,14.454210,-8.868650,0.000000,-2.886975,-0.000625,4.331250,0.461406,0.459688,-0.275000,-0.171875,0.047266


### 表示項目

In [12]:
# "項目名" => 表示subplot-id のdictionary
ncol = 2; nrow = 2
disp_col = {}
disp_col['プレス荷重shift'] = 0
disp_col['右垂直'] = 0
disp_col['スライド変位右'] = 1
# disp_col['M30ボルスタ右奥'] = 3
disp_col['加速度左右_X_左+500G'] = 2

yref = {0:'y', 1:'y2', 2:'y3', 3:'y4'}

In [8]:
nrows = 3; ncols = 2;
disp_col = {}
disp_col['プレス荷重shift'] = [1,1]
disp_col['右垂直'] = [1,1]
disp_col['左垂直'] = [1,1]
disp_col['スライド変位右'] = [2,1]
disp_col['スライド変位左'] = [2,1]
disp_col['M30ボルスタ右奥'] = [1,2]
disp_col['加速度左右_X_左+500G'] = [3,1]
disp_col['加速度上下_Y_下+500G'] = [3,1]
disp_col['加速度前後_Z_前+500G'] = [3,1]
disp_col['スライド＿金型隙間'] = [2,2]


In [8]:
        fig = go.FigureWidget(
            make_subplots(rows=3, cols=2, shared_xaxes=True,vertical_spacing = 0.01,)
        )

In [9]:
fig.layout

Layout({
    'template': '...',
    'xaxis': {'anchor': 'y', 'domain': [0.0, 0.45], 'matches': 'x5', 'showticklabels': False},
    'xaxis2': {'anchor': 'y2', 'domain': [0.55, 1.0], 'matches': 'x6', 'showticklabels': False},
    'xaxis3': {'anchor': 'y3', 'domain': [0.0, 0.45], 'matches': 'x5', 'showticklabels': False},
    'xaxis4': {'anchor': 'y4', 'domain': [0.55, 1.0], 'matches': 'x6', 'showticklabels': False},
    'xaxis5': {'anchor': 'y5', 'domain': [0.0, 0.45]},
    'xaxis6': {'anchor': 'y6', 'domain': [0.55, 1.0]},
    'yaxis': {'anchor': 'x', 'domain': [0.6733333333333333, 1.0]},
    'yaxis2': {'anchor': 'x2', 'domain': [0.6733333333333333, 1.0]},
    'yaxis3': {'anchor': 'x3', 'domain': [0.33666666666666667, 0.6633333333333333]},
    'yaxis4': {'anchor': 'x4', 'domain': [0.33666666666666667, 0.6633333333333333]},
    'yaxis5': {'anchor': 'x5', 'domain': [0.0, 0.32666666666666666]},
    'yaxis6': {'anchor': 'x6', 'domain': [0.0, 0.32666666666666666]}
})

### JupyterDash

In [29]:
import sys
sys.path

['/home/gifu-u/Notebooks/hao',
 '/home/gifu-u/Notebooks/lib',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/home/gifu-u/Workspace/gifu/lib/python3.7/site-packages',
 '/home/gifu-u/Workspace/gifu/lib/python3.7/site-packages/IPython/extensions',
 '/home/gifu-u/.ipython']

In [10]:
from brew_features import *

Loading BokehJS ...

In [11]:
app = make_app()
app.run_server(
#    methode='inline',
#     host='0.0.0.0',
    port=8050,
    debug=True
)

make_app: {'プレス荷重shift': [1, 1], '右垂直': [1, 1], '左垂直': [1, 1], 'スライド変位右': [2, 1], 'スライド変位左': [2, 1], 'M30ボルスタ右奥': [1, 2], '加速度左右_X_左+500G': [3, 1], '加速度上下_Y_下+500G': [3, 1], '加速度前後_Z_前+500G': [3, 1], 'スライド＿金型隙間': [2, 2]}
Dash app running on http://127.0.0.1:8050/


In [23]:
#
import json
import os
from textwrap import dedent as d
import pandas as pd
import numpy as np
from datetime import date, datetime,timedelta
from fft_tools import *
from plotly_utils import *
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import numpy as np

gened_features = {}  # global!!!

def read_logger(f):
    # 9行目以降のメタ情報を読み込むために一旦8行目以降を読み込み
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7] )
    unit = df.iloc[4]
    offset = df.iloc[3]
    calibration = df.iloc[2]
    v_range = df.iloc[1]
    channel = df.iloc[0]
    # 改めて8行目をヘッダにして読み込む
    # df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1).set_index('time')
    # floatのindex作っちゃダメ!!!
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1)
    try:
        df['プレス荷重shift'] = df['プレス荷重'].shift(-640)
        df['dF/dt'] = df['プレス荷重shift'].diff()
        df['F*dF/dt'] = df['プレス荷重shift'] * df['dF/dt']
        if '加速度左右_X_左+' in df.columns:
            df = df.rename({'加速度左右_X_左+':'加速度左右_X_左+500G'},axis=1)
        if '加速度上下_Y_下+' in df.columns:
            df = df.rename({'加速度上下_Y_下+':'加速度上下_Y_下+500G'},axis=1)
        if '加速度前後_Z_前+' in df.columns:
            df = df.rename({'加速度前後_Z_前+':'加速度前後_Z_前+500G'},axis=1)
    except KeyError:
        print('プレス荷重　無し')

    return df,unit,offset,calibration,v_range,channel

# 項目の情報を取得するため1ファイルだけ先に読む。ToDo:前データ項目共通の前提
from pathlib import Path
p = Path(DATA_ROOT+'/20211004ブレークスルー/')
flist = list(sorted(p.glob('*.CSV')))
f = flist[5]
df = read_logger(f)[0][:3000]     # shot#18   下死点が右端にあるやつ

# def gen_input_forms(row_id):
#     return dbc.Col( dcc.Input(id='feature_name%d'%row_id,value=''), width=1,),\
#         dbc.Col( dcc.Dropdown(id='select_col%d'%row_id,value='プレス荷重shift',options=[{'label':str(s),'value':str(s)} for s in df.columns[1:]]), width=2,),\
#         dbc.Col( dcc.Input(id='rolling_width%d'%row_id,value='99'), width=1,),\
#         dbc.Col( dcc.Dropdown(id='find_type%d'%row_id,value='固定',options=[{'label':str(s),'value':str(s)} for s in ['固定','値域','特徴点']]), width=1,),\
#         dbc.Col( dcc.Dropdown(id='low_feature%d'%row_id,value='-',options=[{'label':'-','value':'-'}]), width=1,),\
#         dbc.Col( dcc.Input(id='low_lim%d'%row_id,value='1000'), width=1,),\
#         dbc.Col( dcc.Dropdown(id='up_feature%d'%row_id,value='-',options=[{'label':'-','value':'-'}]), width=1,),\
#         dbc.Col( dcc.Input(id='up_lim%d'%row_id,value='3000'), width=1,),\
#         dbc.Col( dcc.Dropdown(id='find_target%d'%row_id,clearable=False,value='DPT',options=[{'label':'元波形','value':'DPT'},{'label':'速度','value':'VCT'},{'label':'加速度','value':'ACC'}]), width=1,),\
#         dbc.Col( dcc.Dropdown(id='find_dir%d'%row_id,clearable=False,value='MAX',options=[{'label':'MAX','value':'MAX'},{'label':'MIN','value':'MIN'}]), width=1,)


gened_features = {}  # global!!!

def read_logger(f):
    # 9行目以降のメタ情報を読み込むために一旦8行目以降を読み込み
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7] )
    unit = df.iloc[4]
    offset = df.iloc[3]
    calibration = df.iloc[2]
    v_range = df.iloc[1]
    channel = df.iloc[0]
    # 改めて8行目をヘッダにして読み込む
    # df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1).set_index('time')
    # floatのindex作っちゃダメ!!!
    df = pd.read_csv(f,encoding='cp932' ,skiprows=[0,1,2,3,4,5,6,7,9,10,11,12,13], ).rename({'CH名称':'time'},axis=1)
    try:
        df['プレス荷重shift'] = df['プレス荷重'].shift(-640)
        df['dF/dt'] = df['プレス荷重shift'].diff()
        df['F*dF/dt'] = df['プレス荷重shift'] * df['dF/dt']
        if '加速度左右_X_左+' in df.columns:
            df = df.rename({'加速度左右_X_左+':'加速度左右_X_左+500G'},axis=1)
        if '加速度上下_Y_下+' in df.columns:
            df = df.rename({'加速度上下_Y_下+':'加速度上下_Y_下+500G'},axis=1)
        if '加速度前後_Z_前+' in df.columns:
            df = df.rename({'加速度前後_Z_前+':'加速度前後_Z_前+500G'},axis=1)
    except KeyError:
        print('プレス荷重　無し')

    return df,unit,offset,calibration,v_range,channel

# 等口調抽出操作指示のgridの1行を生成   
def gen_input_forms(row_id,fname='',sel_col='',rw=1,llim=0,ulim=0):
    return dbc.Col( dcc.Input(id='feature_name%d'%row_id,value=fname), width=1,),\
        dbc.Col( dcc.Dropdown(id='select_col%d'%row_id,value=sel_col,options=[{'label':str(s),'value':str(s)} for s in df.columns[1:]]), width=2,),\
        dbc.Col( dcc.Input(id='rolling_width%d'%row_id,value=rw), width=1,),\
        dbc.Col( dcc.Dropdown(id='low_find_type%d'%row_id,value='固定',options=[{'label':str(s),'value':str(s)} for s in ['固定','値域>','値域<','特徴点']]), width=1,),\
        dbc.Col( dcc.Dropdown(id='low_feature%d'%row_id,value='-',options=[{'label':'-','value':'-'}]), width=1,),\
        dbc.Col( dcc.Input(id='low_lim%d'%row_id,value=llim), width=1,),\
        dbc.Col( dcc.Dropdown(id='up_find_type%d'%row_id,value='固定',options=[{'label':str(s),'value':str(s)} for s in ['固定','値域>','値域<','特徴点']]), width=1,),\
        dbc.Col( dcc.Dropdown(id='up_feature%d'%row_id,value='-',options=[{'label':'-','value':'-'}]), width=1,),\
        dbc.Col( dcc.Input(id='up_lim%d'%row_id,value=ulim), width=1,),\
        dbc.Col( dcc.Dropdown(id='find_target%d'%row_id,clearable=False,value='DPT',options=[{'label':'元波形','value':'DPT'},{'label':'速度','value':'VCT'},{'label':'加速度','value':'ACC'}]), width=1,),\
        dbc.Col( dcc.Dropdown(id='find_dir%d'%row_id,clearable=False,value='MAX',options=[{'label':'MAX','value':'MAX'},{'label':'MIN','value':'MIN'}]), width=1,)

# 特徴抽出機能のコア部   ToDo: グラフ操作を分離して特徴抽出だけを呼べるように
def locate_feature(df,fig,feature_name,select_col,rolling_width,low_find_type,low_feature,low_lim,up_find_type,up_feature,up_lim,find_target,find_dir):
    global gened_features
    target_i = None

    if feature_name == '':
        return fig

    x_lim = [0,0]   # 初期値
    
    if low_find_type == '固定':
        x_lim[0] = int(low_lim)
    elif low_find_type == '値域>':                      # 指定値より大きい範囲を検索して左端のindexを返す
        sdf = df[(df[select_col]>=float(low_lim))]
        if len(sdf) > 0:
            x_lim[0] = sdf.index[0]
    elif low_find_type == '値域<':                      # 指定値より小さい範囲を検索して左端のindexを返す
        sdf = df[(df[select_col]<=float(low_lim))]
        if len(sdf) > 0:
            x_lim[0] = sdf.index[0]
        # print('value:',x_lim)
    elif low_find_type == '特徴点':
        try:
            x_lim[0] = gened_features[low_feature] + int(low_lim)
            #, gened_features[up_feature] + int(up_lim)]
        except KeyError:
            print('Error')

    if up_find_type == '固定':
        x_lim[1] = int(up_lim)
    elif up_find_type == '値域>':                      # 指定値より大きい範囲を検索して右端のindexを返す
        sdf = df[(df[select_col]>=float(up_lim))]
        if len(sdf) > 0:
            x_lim[1] = sdf.index[-1]
    elif up_find_type == '値域<':                      # 指定値より小さい範囲を検索して右端のindexを返す
        sdf = df[(df[select_col]<=float(up_lim))]
        if len(sdf) > 0:
            x_lim[1] = sdf.index[-1]
        # print('value:',x_lim)
    elif up_find_type == '特徴点':
        try:
            x_lim[1] = gened_features[up_feature] + int(up_lim)
            #, gened_features[up_feature] + int(up_lim)]
        except KeyError:
            print('Error')
            
    target_fig = yref[disp_col[select_col]]

########3 ToDO!!! ###############
    if low_find_type == '値域>':
        plotly_hspan(fig,float(low_lim),float(low_lim)+5,yref=target_fig,fillcolor='green',alpha=.2)
    if low_find_type == '値域<':
        plotly_hspan(fig,float(low_lim),float(low_lim)-5,yref=target_fig,fillcolor='green',alpha=.2)
    if up_find_type == '値域>':
        plotly_hspan(fig,float(up_lim),float(up_lim)+5,yref=target_fig,fillcolor='green',alpha=.2)
    if up_find_type == '値域<':
        plotly_hspan(fig,float(up_lim),float(up_lim)-5,yref=target_fig,fillcolor='green',alpha=.2)

    print('x_lim:',x_lim)
    plotly_vspan(fig,x_lim[0],x_lim[1],yref=target_fig,alpha=.2)
    # 検索対象時系列データの生成
    if find_target == 'DPT':
        target = df[select_col]
    elif find_target == 'VCT':
        target = df[select_col].rolling(int(rolling_width),center=True).mean().diff()
        # print('VCT:')
    elif find_target == 'ACC':
        target = df[select_col].rolling(int(rolling_width),center=True).mean().diff().rolling(int(rolling_width),center=True).mean().diff()
        # print('ACC:')
    if x_lim[1] - x_lim[0] > 0:     # 検索範囲が適切に指定されてなければ何もしない  ToDo:「何もしない」ことのフィードバック? 範囲指定せずに検索したい時もある
#     print('len:',len(target))
        #print('xlim:',x_lim)
        if find_dir == 'MAX':
            target_i = target[x_lim[0]:x_lim[1]].idxmax()
        elif find_dir == 'MIN':
            target_i = target[x_lim[0]:x_lim[1]].idxmin()
        plotly_vspan(fig,target_i,target_i,ftype='line',yref=target_fig,fillcolor='red',alpha=1.0,layer='above',line_width=2)
    target_fig = yref[disp_col[select_col]]
    #plotly_vspan(fig,x_lim[0],x_lim[1],yref=target_fig)
    # 検索対象時系列データの生成
    if find_target == 'DPT':
        target = df[select_col]
    elif find_target == 'VCT':
        target = df[select_col].rolling(int(rolling_width),center=True).mean().diff()
    elif find_target == 'ACC':
        target = df[select_col].rolling(int(rolling_width),center=True).mean().diff().rolling(int(rolling_width),center=True).mean().diff()
    if x_lim[1] - x_lim[0] > 0:     # 検索範囲が適切に指定されてなければ何もしない  ToDo:「何もしない」ことのフィードバック? 範囲指定せずに検索したい時もある
#     print('len:',len(target))
#         print('xlim:',x_lim)
        if find_dir == 'MAX':
            target_i = target[x_lim[0]:x_lim[1]].idxmax()
        elif find_dir == 'MIN':
            target_i = target[x_lim[0]:x_lim[1]].idxmin()
        plotly_vspan(fig,target_i,target_i,ftype='line',yref=target_fig,alpha=1.0)

    if target_i is not None:
        gened_features[feature_name] = target_i

    return fig

def make_app():

    import plotly.express as px
    from jupyter_dash import JupyterDash
    import dash_core_components as dcc
    import dash_bootstrap_components as dbc
    import dash_html_components as html
    from dash.dependencies import Input, Output

#     app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CERULEAN])
    app = JupyterDash('brewFeatures', external_stylesheets=[dbc.themes.CERULEAN])

    # 画面全体のレイアウト
    app.layout = html.Div([
    #     dcc.Dropdown(id='shot_select',options=[{'label':'a','value':'a'},{'label':'b','value':'b'},]),
        # ショット選択
        dbc.Row([
            dbc.Col(
                dcc.Dropdown(id='shot_select',value=str(flist[9]),options=[{'label':str(f), 'value':str(f)} for f in flist[8:]]),
                width=5,
                style={'width':'50vw'} # viewpoint height
    #         style={'height': '20vh','width':'50vw'} # viewpoint height
            ),
        #dbc.Col( dcc.Dropdown(id='find_type1',value='固定',options=[{'label':str(s),'value':str(s)} for s in ['固定','値域','特徴点']]), width=1,),
        ]),
    #     dcc.Dropdown(id='shot_select',options=[{'label':str(f), 'value':str(f)} for f in flist[8:]]),
        # グラフ表示部
        dcc.Graph(id='graph'),
        dbc.Row([
            dbc.Col(dbc.Label('特徴量名')),
            dbc.Col(dbc.Label('対象項目'),width=2),
            dbc.Col(dbc.Label('移動平均範囲')),
            dbc.Col(dbc.Label('下限限定方法')),
            dbc.Col(dbc.Label('下限特徴量')),
            dbc.Col(dbc.Label('下限位置')),
            dbc.Col(dbc.Label('上限限定方法')),
            dbc.Col(dbc.Label('上限特徴量')),
            dbc.Col(dbc.Label('上限位置')),
            dbc.Col(dbc.Label('検索対象')),
            dbc.Col(dbc.Label('検索方向')),
        ]),            
        # 特徴抽出操作指示: gen_input_forms()がInput,Dropdownを含むdbc.Colのリストを生成する
        dbc.Row(gen_input_forms(0,fname='vct_min',sel_col='プレス荷重shift',rw=9,llim=1000,ulim=3000)),
        dbc.Row(gen_input_forms(1)),
        dbc.Row(gen_input_forms(2)),
        dbc.Row(gen_input_forms(3)),
    ])

    # 特徴点セット2行目
    @app.callback(
        Output('low_feature1', 'options'),
        [Input('low_find_type1', 'value'),Input('feature_name0', 'value'),Input('low_feature1', 'options'),
        ])
    def set_low_features1(find_type,fname0,options):
        if find_type == '特徴点':
            return [{'label':fname0, 'value':fname0}]
        else:
            return options
    @app.callback(
        Output('up_feature1', 'options'),
        [Input('up_find_type1', 'value'),Input('feature_name0', 'value'),Input('up_feature1', 'options'),
        ])
    def set_up_features1(find_type,fname0,options):
        if find_type == '特徴点':
            return [{'label':fname0, 'value':fname0}]
        else:
            return options

    # 特徴点セット3行目 (1行目と2行目のfeature_nameをDropdownにセットする。このコーディングどうにかならんか....)
    @app.callback(
        Output('low_feature2', 'options'),
        [Input('low_find_type2', 'value'),Input('feature_name0', 'value'),Input('feature_name1', 'value'),Input('low_feature2', 'options'),
        ])
    def set_low_features2(find_type,fname0,fname1,options):
        if find_type == '特徴点':
            return [{'label':f, 'value':f} for f in [fname0,fname1]]
        else:
            return options
    @app.callback(
        Output('up_feature2', 'options'),
        [Input('up_find_type2', 'value'),Input('feature_name0', 'value'),Input('feature_name1', 'value'),Input('up_feature2', 'options'),
        ])
    def set_up_features2(find_type,fname0,fname1,options):
        if find_type == '特徴点':
            return [{'label':f, 'value':f} for f in [fname0,fname1]]
        else:
            return options

    # 特徴点セット4行目 (1行目と2行目のfeature_nameをDropdownにセットする。このコーディングどうにかならんか....)
    @app.callback(
        Output('low_feature3', 'options'),
        [Input('low_find_type3', 'value'),Input('feature_name0', 'value'),Input('feature_name1', 'value'),Input('feature_name2', 'value'),Input('low_feature3', 'options'),
        ])
    def set_low_features3(find_type,fname0,fname1,fname2,options):
        if find_type == '特徴点':
            return [{'label':f, 'value':f} for f in [fname0,fname1,fname2]]
        else:
            return options      # '特徴点'以外の場合は現状のまま...で良いのか?
    @app.callback(
        Output('up_feature3', 'options'),
        [Input('up_find_type3', 'value'),Input('feature_name0', 'value'),Input('feature_name1', 'value'),Input('feature_name2', 'value'),Input('up_feature3', 'options'),
        ])
    def set_up_features3(find_type,fname0,fname1,fname2,options):
        if find_type == '特徴点':
            return [{'label':f, 'value':f} for f in [fname0,fname1,fname2]]
        else:
            return options
        
    @app.callback(
#     dash.dependencies.Output('graph', 'figure'),
        Output('graph', 'figure'),
        [Input('shot_select', 'value'),
             Input('feature_name0','value'),Input('select_col0','value'),Input('rolling_width0','value'),Input('low_find_type0','value'),Input('low_feature0','value'),Input('low_lim0','value'),
             Input('up_find_type0','value'),Input('up_feature0','value'),Input('up_lim0','value'),Input('find_target0','value'),Input('find_dir0','value'),
         Input('feature_name1', 'value'),
             Input('select_col1', 'value'),Input('rolling_width1', 'value'),Input('low_find_type1', 'value'),Input('low_feature1', 'value'),Input('low_lim1','value'),
             Input('up_find_type1','value'),Input('up_feature1', 'value'),Input('up_lim1', 'value'),Input('find_target1', 'value'),Input('find_dir1', 'value'),
         Input('feature_name2', 'value'),
             Input('select_col2', 'value'),Input('rolling_width2', 'value'),Input('low_find_type2', 'value'),Input('low_feature2', 'value'),Input('low_lim2','value'),
             Input('up_find_type2','value'),Input('up_feature2', 'value'),Input('up_lim2', 'value'),Input('find_target2', 'value'),Input('find_dir2', 'value'),
         Input('feature_name3', 'value'),
             Input('select_col3', 'value'),Input('rolling_width3', 'value'),Input('low_find_type3', 'value'),Input('low_feature3', 'value'),Input('low_lim3','value'),
             Input('up_find_type3','value'),Input('up_feature3', 'value'),Input('up_lim3', 'value'),Input('find_target3', 'value'),Input('find_dir3', 'value'),
        ])
    def callback_figure(shot,
            feature_name0,select_col0,rolling_width0,low_find_type0,low_feature0,low_lim0,up_find_type0,up_feature0,up_lim0,find_target0,find_dir0,
            feature_name1,select_col1,rolling_width1,low_find_type1,low_feature1,low_lim1,up_find_type1,up_feature1,up_lim1,find_target1,find_dir1,
            feature_name2,select_col2,rolling_width2,low_find_type2,low_feature2,low_lim2,up_find_type2,up_feature2,up_lim2,find_target2,find_dir2,
            feature_name3,select_col3,rolling_width3,low_find_type3,low_feature3,low_lim3,up_find_type3,up_feature3,up_lim3,find_target3,find_dir3,
            ):
        gened_features = {}
        #print(shot)
        df = read_logger(shot)[0][:4000]
#         fig = multi_col_figure(df[['プレス荷重shift','スライド変位右','加速度左右_X_左+500G']])
        fig = go.FigureWidget(
            make_subplots(rows=max(disp_col.values())+1, cols=1, shared_xaxes=True,vertical_spacing = 0.01,)
        )
        for col in disp_col:
            fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col), row=disp_col[col]+1, col=1)
        fig.update_layout(showlegend=True, title_text='shot No.', width=1600,height=800)

        # ToDo: 今のところ、gridの1行ごとにlocate_featureが必要
        fig = locate_feature(df,fig,feature_name0,select_col0,rolling_width0,low_find_type0,low_feature0,low_lim0,up_find_type0,up_feature0,up_lim0,find_target0,find_dir0)
        fig = locate_feature(df,fig,feature_name1,select_col1,rolling_width1,low_find_type1,low_feature1,low_lim1,up_find_type1,up_feature1,up_lim1,find_target1,find_dir1)
        fig = locate_feature(df,fig,feature_name2,select_col2,rolling_width2,low_find_type2,low_feature2,low_lim2,up_find_type2,up_feature2,up_lim2,find_target2,find_dir2)
        fig = locate_feature(df,fig,feature_name3,select_col3,rolling_width3,low_find_type3,low_feature3,low_lim3,up_find_type3,up_feature3,up_lim3,find_target3,find_dir3)        

        return fig


    return app

# if __name__ == '__main__':
#     app = make_app()
app = make_app()
# app.run_server(mode='inline',host='0.0.0.0',port=8048,debug=True)
app.run_server(
#    methode='inline',
#     host='0.0.0.0',
    port=8050,debug=True
)
# app.run_server(mode='inline',debug=True)   # ==> localhost:8050

Dash app running on http://127.0.0.1:8050/
x_lim: [1000, 3000]
x_lim: [1000, 3000]


これだとaddress already in use
```
app.run_server(
#    methode='inline',
    host='0.0.0.0',
    port=8050,debug=True
)
```

```
~/Workspace/gifu/lib/python3.7/site-packages/retrying.py in call(self, fn, *args, **kwargs)
    198         while True:
    199             try:
--> 200                 attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
    201             except:
    202                 tb = sys.exc_info()

~/Workspace/gifu/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py in wait_for_app()
    313                     "Address '{url}' already in use.\n"
    314                     "    Try passing a different port to run_server.".format(
--> 315                         url=url
    316                     )
    317                 )

OSError: Address 'http://0.0.0.0:8050' already in use.
    Try passing a different port to run_server.
```    

これなら起動はできる。が、remoteからはアクセスできない。VNCとか使えば行けるだろうけどね...
```
#    methode='inline',
#     host='0.0.0.0',
    port=8050,debug=True
)
```

```
Dash app running on http://127.0.0.1:8050/
```

コンソールからなら使えるんだろうから、最悪どうにかなるか? どのみちdockerだとダメか?
```
tcp        0      0 127.0.0.1:8050          0.0.0.0:*               LISTEN
```

VNC入れるよりこっちの方が楽そうだったので、apacheでリバプロ立てたら行けた。  
jupyterdash.quattro.unisys.co.jp -> localhost:8050に転送。
```
NameVirtualHost 10.26.12.223:80
<VirtualHost 10.26.12.223:80>
        ServerName jupyterdash.quattro.unisys.co.jp
        ServerAdmin postmaster@jupyter.quattro.unisys.co.jp
        CustomLog /var/log/apache2/jupyter.log combined
        ErrorLog /var/log/apache2/jupyter-error.log
        ProxyRequests Off
        ProxyPass               /robots.txt     http://127.0.0.1/robot/robo.txt
        ProxyPass           /api/kernels/ ws://127.0.0.1:8050/api/kernels/
        ProxyPassReverse    /api/kernels/ ws://127.0.0.1:8050/api/kernels/
        ProxyPass           /terminals/websocket/ ws://127.0.0.1:8050/terminals/websocket/
        ProxyPassReverse    /terminals/websocket/ ws://127.0.0.1:8050/terminals/websocket/
        ProxyPass               /       http://127.0.0.1:8050/
        ProxyPassReverse        /       http://127.0.0.1:8050/
```

In [8]:
disp_col

{'プレス荷重shift': 0, '右垂直': 0, 'スライド変位右': 1, '加速度左右_X_左+500G': 2}

In [52]:
row = html.Div(
    [
        dbc.Row(dbc.Col(html.Div("A single column"))),
        dbc.Row(
            [
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
                dbc.Col(html.Div("One of three columns")),
            ]
        ),
    ]
)

In [53]:
row

Div([Row(Col(Div('A single column'))), Row([Col(Div('One of three columns')), Col(Div('One of three columns')), Col(Div('One of three columns'))])])